In [2]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
import ast
from fractions import Fraction
import warnings
warnings.filterwarnings('ignore')

In [3]:
# parameters
customer_id = '5'
formatted_attribute = 'Cord Length'
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')

# buckets = """Window Blinds"""
buckets = """All_Buckets"""

regex_pattern=r'(?i)(cord length measures\s?\d+.?\d*.?\")|((cord length measures\s?\d+.?\d*.?ft))()'
regex_pattern_inch=r'(?i)(cord length measures\s?\d+.?\d*.?\")|(cord measures\s?\d+.?\d*.?\")|()'
regex_pattern_foot=r"(?i)((cord length measures\s?\d+.?\d*.?ft))|((cord length measures\s?\d+.?\d*.?\'))|((cord measures\s?\d+.?\d*.?ft))|((cord measures\s?\d+.?\d*.?\'))|()"


value='%n/a%'
customer_name='%bedbathandbeyond%'

params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [4]:
def query_from_file(file_name, params):
    with open(f'{file_name}', mode='r') as f:
        text = f.read()
        query = text.format(**params)
        return enrich_db(query)

In [5]:
# df = query_from_file(file_name='Curate_wipe/BusinessIntelligence/Platform Queries/Bucket Attribute Values for All active Strategies.sql', params=params)
df = query_from_file(file_name='query/uncurated.sql', params=params)
# df

In [6]:
df[df['external_id'].astype(str)=='44688578']

product_id external_id                             product_name  \
5571   817662246    44688578  Bionaire® Ceramic Quartz Console Heater   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 long_desc  \
5571  Easily keep your living space the temperature you want with the Bionaire Ceramic Quartz Console Heater. This small but powerful heater features easy-to-use digital controls and a cool touch exterior, making house-heating more convenient than ever before. Keep warm with the Bionaire Ceramic Quartz Console Heater Encased infrared radiant technology diffuses heat 3-speed control for personal preference Safe cool touch exterior 1,500 watts of power Digital control panel with LED display Sturdy plastic construction Measures 18.75\" L x 13.5\" W x 17.5\" H 1-year manufacturer's warranty Imported   

      customer_id attributes            buckets values  
5571            5       None  Heating Equipment   None

In [8]:
def re_extract(pattern, txt):
    matches = re.findall(pattern, txt)
    tmp_matches = []
    for match in matches:
        for tup in match:
            if tup != '':
                tmp_matches.append(tup)
    return list(set(tmp_matches))

In [9]:
df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
# df

In [11]:
def clean_data(data):
    data['matches']=data['matches'].str.get(0)
#     data=data.dropna()
    data['matches']=data['matches'].apply(lambda x:x.lower())

In [12]:
df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_inch, x))
inch=df[df['matches'].astype(str)!='[]']
clean_data(inch) #might throw an error and have to drop nane
inch['matches']=inch['matches'].apply(lambda measure: re.sub(r"[a-z]|[A-z]|\s|\w*\.\w*\.|\-|\'\\",'',str(measure))).replace("'", '').apply(lambda quote: quote.replace('"', ''))
# inch

In [13]:
df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_foot, x))
foot=df[df['matches'].astype(str)!='[]']
clean_data(foot) #might throw an error and have to drop nane
foot['matches']=foot['matches'].apply(lambda measure: re.sub(r"[a-z]|[A-z]|\s|\w*\.\w*\.|\-|\\|\'",'',str(measure))).replace("'", '').apply(lambda quote: quote.replace('"', ''))
foot['matches']=foot['matches'].astype(float).apply(lambda x: float(x*12)).astype(int)
# foot

In [ ]:
matches=pd.concat([inch,foot])
matches

In [ ]:
x=pd.DataFrame()
x['col']=[1.00, 1.10, 1.02, 1.003]
x['col']=x['col'].astype(str).apply(lambda quote: quote.replace('.0', ''))
x

In [ ]:
matches[curation_col] = matches['matches'].astype(str)+' in'
# matches['external_id']=matches['external_id'].apply('="{}"'.format)
matches

In [ ]:
df=df[df['matches'].astype(str)!='[]']
df

In [ ]:
df['matches'].explode().value_counts()

In [ ]:
def split_col(data):
    data['matches']=data['matches'].str.get(0)
#     data=data.dropna()
    data['matches']=data['matches'].apply(lambda x:x.lower())
#     data[['length', 'width']]=data['matches'].str.split(" x ",expand=True,)

In [ ]:
split_col(df) #might throw an error and have to drop nane

In [ ]:
df

In [ ]:
def drop_characters(data):
    data['matches']=data['matches'].apply(lambda measure: re.sub(r"[a-z]|[A-z]|\s|\w*\.\w*\.|\-|\'",'',str(measure)))
#     data['width']=data['width'].apply(lambda measure: re.sub(r"[a-z]|[A-z]|\s|\w*\.\w*\.|\-|\'",'',str(measure)))
    data['matches']=data['matches'].replace("'", '').apply(lambda quote: quote.replace('"', ''))
#     data['width']=data['width'].apply(lambda quote: quote.replace('"', ''))
    data['matches']=data['matches'].replace("'", '').apply(lambda quote: quote.replace('-',''))
#     data['width']=data['width'].apply(lambda quote: quote.replace('-',''))

In [ ]:
drop_characters(df)

In [ ]:
df

In [ ]:
def replace_none(data):
#     data['width']=data['width'].apply(lambda x: 0 if x == '' else x)
    data['matches']=data['matches'].apply(lambda x: 0 if x == '' else x)
#     data['width']=data['width'].apply(lambda x: 0 if x == 'None' else x)
    data['matches']=data['matches'].apply(lambda x: 0 if x == 'None' else x)

In [ ]:
def add_quotes(data):
    data['matches']=data['matches'].apply(lambda x: str(x) + "'")
#     data['width']=data['width'].apply(lambda x: str(x) + "'")



In [ ]:
# def min_max(data):
#     data['length']=data['length'].astype(float)
#     data['width']=data['width'].astype(float)
#     data['Max'] = data[['length', 'width']].values.max(1)
#     data['Min'] = data[['length', 'width']].values.min(1)

In [ ]:
def remove_fractions(numbers):
    if numbers.find('-') > -1:
        numbers = literal_eval(numbers)
        inte, fract = numbers.split(' ')
        nom, den = fract.split('/')
        tmp = float(nom)/float(den)
        num = int(inte) + tmp
        return str(num)
    else:
        return numbers

In [ ]:
split_col(df) #might throw an error and have to drop nane

In [ ]:
# df

In [ ]:
# df['width']=df['width'].astype(str).apply(lambda x: 0 if x == 'nan' else x)
# df

In [ ]:
# df['length']=df['length'].astype(str)
# df['width']=df['width'].astype(str)
# df['length']=df['length'].apply(lambda quote: quote.replace('-', ' '))
# df['width']=df['width'].apply(lambda quote: quote.replace('-', ' '))
# df['length']=df['length'].apply(lambda quote: quote.replace('inch', ''))
# df['width']=df['width'].apply(lambda quote: quote.replace('inch', ''))
# df

In [ ]:
# df['length']=df['length'].apply(lambda measure: re.sub(r"[a-z]|[A-z]|\w*\.\w*\.|\-|\'",'',str(measure)))
# df['width']=df['width'].apply(lambda measure: re.sub(r"[a-z]|[A-z]|\w*\.\w*\.|\-|\'",'',str(measure)))
# df

In [ ]:
error----Switch Max and Min when doing length and width

In [ ]:
gc = gcloud.Gsheets()

In [ ]:
gc.save_to_gsheet('https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0','ul',matches)

# send to the folder for upload

In [ ]:
def looks_good(customer, attribute, df, matches):
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload/Shawn'
    non_matches = df[df['matches'].astype(str) == '[]']
    non_matches[curation_col] = r'n/a'
    non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/BBBY - {attribute} - match upload {buckets}.csv',index=False)

In [ ]:
looks_good('Bed Bath & Beyond', attribute, df, matches)